# CAGRA Example

This notebook is adapted from the [Milvus 2.4 examples](https://github.com/milvus-io/pymilvus/blob/2.4/examples/example_gpu_cagra.py). Running this example code requires the deployment of a Milvus instance that attached with GPU devices. See `milvus-setup.ipynb` for starting Milvus 2.4 with GPU support in stand-alone mode.

In [1]:
import numpy as np
import time 
import random

from pymilvus import (
    connections,
    list_collections,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

dim = 128
nb = 10000
collection_name = "GPU_CAGRA_test"

## Create Connection

In [2]:
connections.connect(host="localhost", port=19530)

## List all collection names

In [3]:
collection_list = list_collections()
print(list_collections())

if(collection_list.count(collection_name)):
    print(collection_name, " exist, and drop it")
    collection = Collection(collection_name)
    collection.drop()
    print("drop")

field1 = FieldSchema(name="id", dtype=DataType.INT64, description="int64", is_primary=True)
field2 = FieldSchema(name = "vec", dtype = DataType.FLOAT_VECTOR, description = "float vector", dim = dim, is_primary = False)
schema = CollectionSchema(fields = [field1, field2], description = "sift decription")
collection = Collection(name = collection_name, data = None, schema = schema, shards_num = 2)

print(list_collections())
print(collection.partitions)

['VectorDBBenchCollection', 'GPU_CAGRA_test']
GPU_CAGRA_test  exist, and drop it
drop
['VectorDBBenchCollection', 'GPU_CAGRA_test']
[{"name":"_default","collection_name":"GPU_CAGRA_test","description":""}]


## Begin insert

In [4]:
rng = np.random.default_rng(seed=19530)
data = rng.random((nb, dim))
counter = 0
block_num = 100
block_size = int(data.shape[0]/block_num)

start = time.time()
for t in range(block_num):
    entities = [
            [i for i in range(counter, counter + block_size)],
            # [vectors[i] for i in range(counter, counter + block_size)]
            [vec for vec in data[counter: counter + block_size]]
            ]
    insert_result =  collection.insert(entities)
    counter = counter + block_size
print ("end of insert, cost: ", time.time()-start)

end of insert, cost:  0.670280933380127


In [5]:
collection.flush()
print(collection.num_entities)

10000


## Create index

In [6]:
collection.create_index(field_name="vec",
        index_params={'index_type': 'GPU_CAGRA',  
            'metric_type': 'L2',
            'params': {
                'intermediate_graph_degree':64,
                'graph_degree': 32,
                'build_algo': 'NN_DESCENT'
                }})

Status(code=0, message=)

## Load Data

In [7]:
collection.load()

## Search

In [8]:
print(f"\nSearch...")
res = collection.search([vec for vec in data[0:1]],
                        "vec", 
                            {"metric_type": "L2",
                              "params": {
                                   "search_width":100},
                                   }, 
                                limit=100)
print("run result: ", res[0].ids)


Search...
run result:  [0, 497, 7194, 5511, 8201, 8728, 1564, 5025, 266, 2660, 8890, 486, 3486, 6512, 6494, 3289, 8658, 9383, 2478, 4640, 1123, 9216, 1263, 7240, 190, 6387, 6289, 5156, 6654, 9823, 2535, 2350, 9205, 7729, 1091, 898, 3334, 9982, 2183, 7434, 996, 8485, 8011, 3150, 6245, 3215, 1147, 9734, 500, 9232, 5129, 1054, 415, 554, 9012, 7653, 4678, 8562, 9055, 8155, 479, 3781, 6807, 6047, 7975, 7623, 9144, 807, 3186, 6453, 875, 4736, 275, 1977, 3147, 1769, 8654, 146, 1614, 9918, 8608, 334, 2894, 2697, 7928, 4683, 4639, 8188, 6071, 8126, 9800, 5547, 1659, 1974, 9646, 4382, 4805, 5086, 3545, 4987]


In [9]:
collection.drop()